Welcome to day 1 of the 5-Day Data Challenge! Today, we're going to be looking at how to deal with missing values. To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

> **Your turn!** As we work through this notebook, you'll see some notebook cells (a block of either code or text) that has "Your Turn!" written in it. These are exercises for you to do to help cement your understanding of the concepts we're talking about. Once you've written the code to answer a specific question, you can run the code by clicking inside the cell (box with code in it) with the code you want to run and then hit CTRL + ENTER (CMD + ENTER on a Mac). You can also click in a cell and then click on the right "play" arrow to the left of the code. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

# 데이터 전체를 보는 방법은 기본적으로 같습니다.
### 모듈을 import합니다. # pandas와 numpy를 썼습니다.
### np.random.seed(0)?
* 이게 어떤 의미인지 설명드리도록 하겠습니다.
* 우선 참고 사이트를 읽어보시기 바랍니다. <https://datascienceschool.net/view-notebook/8bf41f87a08b4c44b307799577736a28/>
* 즉 난수 발생의 무작위성을 제어하기 위한 장치라고 생각하시면 되겠습니다. 난수가 계속 바뀌면 코드를 실행한 결과가 사람들마다 다르게 나타날 수 있기 때문에 
난수를 발생하되 발생하는 난수를 제어해 일정한 값을 보일 수 있도록 하는 것 입니다.

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("../input/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0) 

The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

# 데이터를 살펴봅시다.
* 지정된 변수명 .sample(number) : 지정된 변수명의 구성 요소들을 숫자만큼 보여준다.
* nfl_data.sample(1)을 하며 low 1로 인식되어 1줄만 출력됩니다.

In [ ]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
nfl_data.sample(5)

Yep, it looks like there's some missing values. What about in the sf_permits dataset?

In [ ]:
# your turn! Look at a couple of rows from the sf_permits dataset. Do you notice any missing data?
sf_permits.sample(5)
# your code goes here :)

# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

# 없거나 기록되지 않은 자료를 찾아보는 시간
* 데이터를 살펴볼 때 모든 값들이 다 들어가 있으면 좋겠지만, 그럴리는 없습니다. 자비 따윈 없지요.
* 그렇기 때문에 어디가 구멍이 나있는지 찾아야 하고 그 빈 값을 어떻게 찾을 것인가 고민해 봅시다.
* 사용할 명령어는 변수명.isnull().sum() 입니다.
* 각 low 값들이 몇 개의 null을 가지고 있는지 알기 위해 지정된 변수명[0:10] or [:] (전체 다 보기)

In [ ]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

# 비율
* 전체 셀에 대해서 missing 값은 얼만큼 있나를 찾는 시간입니다.
* np.product가 나오는데 이 부분은 추후 np를 설명할 때 알려드리도록 하겠습니다.(사실 행렬 공부해야 해서 당사자도 잘 몰라요 ....흑)
* 나머지 작업은 수식 보시면 이해가 되실 겁니다.

In [ ]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [ ]:
# your turn! Find out what percent of the sf_permits dataset is missing
missing_values_count_sf = sf_permits.isnull().sum()
missing_values_count_sf[0:10]

In [ ]:
total_cell_count_sf = np.product(sf_permits.shape)
total_miising_cell_count_sf = missing_values_count_sf.sum()
(total_miising_cell_count_sf/total_cell_count_sf)*100

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

* 이 글을 읽는 분들의 영어 실력을 믿습니다.^^
* 간단히 정리하면 NaN으로 존재하는 것은 의미없는 것을 추축하는 것 자체가 의미가 없기 때문에 NaN으로 나타나지만, 기록되지 않아 값이 누락된 경우 다른 행, 열을 기반으로 값을 찾을 수도 있습니다.(이것을 imputation이라고 하며 아래에서 배웁니다.)
* 결국 정리하면 비어있다고 꼭 그 값이 의미를 지니는 것은 아니며, 기록되지 않았다고 해서 의미를 못 찾는 것도 아니겠습니다.

In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

# missing values 값 날려버리기
* 값이 포함된 row를 날려버리도록 하겠습니다.
* 사용되는 함수는 pandas에 dropna()를 쓰겠습니다.

In [ ]:
# remove all the rows that contain a missing value
nfl_data.dropna()

* 그런데 위에 처럼 다 날려버리면 볼 게 없으니 슬프겠죠.
* 그러니 우리는 특정 조건을 설정해서 필요한 부분만 날리도록 하겠습니다.
* 여기서 필요한 개념이 axis에 대한 개념인데, axis = 1 은 columns에 해당한다고 생각하세요.(자세한 건 따로 설명 드립니다.)

Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

* %d 는 입력 할 사항이 길어서 한 번에 다 쓰기 번거로우니 뒤로 빼서 작성할 때 사용해주곤 합니다.

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [ ]:
# Your turn! Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?
sf_permits.dropna()

In [ ]:
columns_with_na_dropped_sf = sf_permits.dropna(axis=1)
columns_with_na_dropped_sf.head()

In [ ]:
print("Columns in original dataset: %d \n" % sf_permits.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped_sf.shape[1])

<https://stackoverflow.com/questions/22149584/what-does-axis-in-pandas-mean>
axis = 1 에 대한 설명
<http://taewan.kim/post/numpy_sum_axis/> 좀 더 수학이면서 한글 설명

In [ ]:
# Now try removing all the columns with empty values. Now how much of your data is left?
columns_with_na_dropped_sf = sf_permits.dropna(axis=1)
columns_with_na_dropped_sf

# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

* loc는 인덱서입니다.
* loc 인덱서를 사용할 때는 행/열 인덱스들이 모두 다음 중 하나이어야 한다.

    1) 정수 인덱스가 아닌 라벨 값(원래 인덱스 자체가 정수 인덱스인 경우는 예외적으로 허용)

    2) 라벨 값의 리스트나 슬라이싱

    3) 불리언 리스트, 1차원 배열, 시리즈 (데이터프레임은 안된다.)

    4) 또는 데이터프레임을 입력으로 받고 위의 값을 반환하는 함수
  
  <https://datascienceschool.net/view-notebook/704731b41f794b8ea00768f5b0904512/> 참고하세요.
  

* 인덱서를 활용해 'EPA' ~ 'Season' 에 해당하는 값을 찾을 겁니다.

In [ ]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data.head()

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

쉽게 말하면 NaN 을 0으로 치환하겠다. 

응용하면 na 영역에 평균을 넣던 다른 숫자를 넣을 수도 있겠습니다.

In [ ]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna("0")

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?

In [ ]:
# Your turn! Try replacing all the NaN's in the sf_permits data with the one that
subset_sf_permits = sf_permits.loc[:, 'Street Number Suffix':'Existing Construction Type'].head()
subset_sf_permits
# comes directly after it and then 

In [ ]:
subset_sf_permits.fillna(0)

In [ ]:
subset_sf_permits.fillna(method = 'bfill', axis=0).fillna("X")

And that's it for today! If you have any questions, be sure to post them in the comments below or [on the forums](https://www.kaggle.com/questions-and-answers). 

Remember that your notebook is private by default, and in order to share it with other people or ask for help with it, you'll need to make it public. First, you'll need to save a version of your notebook that shows your current work by hitting the "Commit & Run" button. (Your work is saved automatically, but versioning your work lets you go back and look at what it was like at the point you saved it. It also let's you share a nice compiled notebook instead of just the raw code.) Then, once your notebook is finished running, you can go to the Settings tab in the panel to the left (you may have to expand it by hitting the [<] button next to the "Commit & Run" button) and setting the "Visibility" dropdown to "Public".

# More practice!
___

If you're looking for more practice handling missing values, check out these extra-credit\* exercises:

* [Handling Missing Values](https://www.kaggle.com/dansbecker/handling-missing-values): In this notebook Dan shows you several approaches to imputing missing data using scikit-learn's imputer. 
* Look back at the `Zipcode` column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset. You can search for datasets about San Fransisco on the [Datasets listing](https://www.kaggle.com/datasets).) 

\* no actual credit is given for completing the challenge, you just learn how to clean data real good :P